# Wavelength Loss dependence using BraggExact

Here for the N2 configuration from Bird.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from scipy.optimize import newton

# Initialize

In [ ]:
n = 300
wls = np.linspace(.4e-6, 1.6e-6, n+1)
betas1 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

In [ ]:
# n_air = 1.00027717
# n_glass = 1.4388164768221814
# ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass]
# ts = [15*1e-6, .42*1e-6, 15*1e-6, 15*1e-6]

In [ ]:
ts = [15*1e-6, .42*1e-6, 15*1e-6, 10*1e-6, 5e-6]
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass, lambda x:n_air]
mats = ['air', 'glass', 'air', 'glass','air']

# Four Layer Study: Glass outside Air (Outside Cladding)

In [ ]:
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=[100 ,100, 100, 100, 100], wl=wl,no_mesh=True)

    k_low = A.ks[0] * A.scale
    if wl <.8e-6:
        guess = np.array(.99995 * k_low)
    else:
        guess = np.array(.9999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas1[i] = beta
    

In [ ]:
# Formula for loss spikes from article

ms1 = np.arange(1,3,1)
ms2 = np.arange(13,60,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d1 = ts[1]
d2 = ts[3]

ls1 = (2 * n1 * d1 / ms1 * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver
ls2 = (2 * n1 * d2 / ms2 * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver


In [ ]:
%matplotlib notebook
plt.figure(figsize=(14,7))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9)
plt.yscale('log')
m, M = plt.ylim()
for l in ls1:
    plt.plot([l,l], [m, M], color='red', linewidth=1, linestyle='--')
    
for l in ls2:
    if l >= wls[0] and l <= wls[-1]:
        plt.plot([l,l], [m, M], color='grey',  linewidth=1, linestyle=':')

# Base

In [ ]:
n = 300
wls = np.linspace(.4e-6, 1.6e-6, n+1)
betas2 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass]
ts = [15*1e-6, .42*1e-6, 15*1e-6, 15*1e-6]
mats = ['air', 'glass', 'air', 'glass']

# Four Layer Study: Glass outside Air (Outside Cladding)

In [ ]:
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=[100 ,100, 100, 100], wl=wl,no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas2[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(14,7))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9)
plt.plot(wls, -betas2.imag, color='firebrick', linewidth=.9)

plt.yscale('log')
m, M = plt.ylim()
for l in ls1:
    plt.plot([l,l], [m, M], color='red', linewidth=1, linestyle='--')
    
    
for l in ls2:
    if l >= wls[0] and l <= wls[-1]:
        plt.plot([l,l], [m, M], color='grey',  linewidth=1, linestyle=':')

## Bad wavelengths

In [ ]:
ls
np.where(abs(wls-1.88e-6)<(wls[1]-wls[0]))

In [ ]:
wls[240]

In [ ]:
A = BraggExact(ts=ts, ns=ns, maxhs=[.1 ,.02, .04, .02], wl=wls[240])


In [ ]:
k_low = A.ks[0] * A.scale


In [ ]:
plotlogf(A.determinant, .9998*k_low, k_low, -.0015,.0015, nu, outer,
         iref=100, rref=100, levels=100, figsize=(8,6))

In [ ]:
guess = np.array(50.1448)
imag = 0
x0 = np.array(guess + imag*1j)

beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
beta

In [ ]:
betas1[240] = beta

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9)
plt.yscale('log')
for l in ls:
    plt.plot([l,l], [0, .0001],  linewidth=1, linestyle=':')

# Using Sellmeier

In [ ]:
betas2=np.zeros_like(betas1)
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, maxhs=[1 ,1, 1, 1], wl=wl)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas2[i] = beta
    

In [ ]:
ls_sell=np.load('ls_sell.npy')

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betas1.imag, color='green', linewidth=.9, label='no sellmeier')
plt.plot(wls, -betas2.imag, color='red', linewidth=.9, label='sellmeier')
plt.yscale('log')
plt.legend()
for l in ls_sell:
    plt.plot([l,l], [0, .0001],  linewidth=1, color='r',linestyle=':')
for l in ls:
    plt.plot([l,l], [0, .0001],  linewidth=1, color='g',linestyle=':')
